In [54]:
import pandas as pd
import datetime as dt
import csv

In [55]:
data_jan = pd.read_csv('../data/raw/cycling_jan15-31.csv')
data_jan['season'] = 'winter'
data_jul = pd.read_csv('../data/raw/cycling_jul15-31.csv')
data_jul['season'] = 'summer'
data = pd.concat([data_jan, data_jul], axis=0, ignore_index=True)

In [56]:
# remove rows with start and end stations overlapping.
data = data[data['Start station number'] != data['End station number']]
data.shape

(840436, 12)

In [57]:
data_dates = pd.to_datetime(data['Start date'])

today_date = dt.datetime.today()
today_day = today_date.weekday()

def next_date_same_weekday(original_date):
    target_weekday = original_date.weekday()
    days_ahead = (target_weekday - today_day + 7) % 7
    new_date = today_date + dt.timedelta(days=days_ahead)
    return new_date.strftime('%Y%m%d')

def workday_or_chillday(original_date):
    target_weekday = original_date.weekday()

    if target_weekday < 5:
        return 'workday'
    elif 5 <= target_weekday <= 7:
        return 'weekend'
    else:
        return None

data['Type of day'] = data_dates.apply(workday_or_chillday)
data['New Start date'] = data_dates.apply(next_date_same_weekday)
data['Start time'] = data_dates.dt.strftime('%H%M')
data.head()

,Number,Start date,Start station number,Start station,End date,End station number,End station,Bike number,Bike model,Total duration,Total duration (ms),season,Type of day,New Start date,Start time
0,146028038,2025-01-31 23:59,1142,"Tooley Street, Bermondsey",2025-02-01 00:35,988,"Great Russell Street, Bloomsbury",22507,CLASSIC,36m 8s,2168663,winter,workday,20250530,2359
1,146028039,2025-01-31 23:59,1142,"Tooley Street, Bermondsey",2025-02-01 00:35,988,"Great Russell Street, Bloomsbury",22066,CLASSIC,36m 14s,2174536,winter,workday,20250530,2359
2,146028040,2025-01-31 23:59,300061,"Crimscott Street, Bermondsey",2025-02-01 00:19,300024,"Dunston Road , Haggerston",53875,CLASSIC,20m 11s,1211944,winter,workday,20250530,2359
3,146028041,2025-01-31 23:59,300058,"The Vale, Chelsea",2025-02-01 00:03,300076,"Lots Road, West Chelsea",54808,CLASSIC,4m 32s,272960,winter,workday,20250530,2359
4,146028042,2025-02-01 00:00,300038,"Star Road, West Kensington",2025-02-01 00:12,300037,"Ravenscourt Park Station, Hammersmith",51308,CLASSIC,12m 54s,774703,winter,weekend,20250531,0000


In [58]:
# remaking dates
data = data.drop(['Number', 'Bike number', 'Total duration', 'End date', 'Start station', 'End station'], axis=1)
data['Total duration (ms)'] = pd.to_numeric(data['Total duration (ms)']) // (1000 * 60)
data = data.rename(columns={'Total duration (ms)': 'Total duration (m)'})
data.head()

,Start date,Start station number,End station number,Bike model,Total duration (m),season,Type of day,New Start date,Start time
0,2025-01-31 23:59,1142,988,CLASSIC,36,winter,workday,20250530,2359
1,2025-01-31 23:59,1142,988,CLASSIC,36,winter,workday,20250530,2359
2,2025-01-31 23:59,300061,300024,CLASSIC,20,winter,workday,20250530,2359
3,2025-01-31 23:59,300058,300076,CLASSIC,4,winter,workday,20250530,2359
4,2025-02-01 00:00,300038,300037,CLASSIC,12,winter,weekend,20250531,0000


In [59]:
stations_data = pd.read_xml('../data/raw/livecyclehireupdates.xml')
stations_data['station_coordinates'] = stations_data['lat'].astype(str) + ',' + stations_data['long'].astype(str)
stations_data = stations_data.drop(['id', 'name', 'lat', 'long', 'installed', 'locked', 'installDate', 'removalDate', 'temporary', 'nbBikes', 'nbStandardBikes', 'nbEBikes', 'nbEmptyDocks', 'nbDocks'], axis=1)
stations_data.head()

,terminalName,station_coordinates
0,1023,"51.52916347,-0.109970527"
1,1018,"51.49960695,-0.197574246"
2,1012,"51.52128377,-0.084605692"
3,1013,"51.53005939,-0.120973687"
4,3420,"51.49313,-0.156876"


In [60]:
merged_data = data.merge(
    stations_data,
    left_on='Start station number',
    right_on='terminalName'
)
merged_data = merged_data.rename(columns={'station_coordinates': 'start_coordinates'})

merged_data = merged_data.merge(
    stations_data,
    left_on='End station number',
    right_on='terminalName'
)
merged_data = merged_data.rename(columns={'station_coordinates': 'end_coordinates'})

merged_data = merged_data.drop(['terminalName_x', 'terminalName_y'], axis=1)
merged_data.head()

,Start date,Start station number,End station number,Bike model,Total duration (m),season,Type of day,New Start date,Start time,start_coordinates,end_coordinates
0,2025-01-31 23:59,1142,988,CLASSIC,36,winter,workday,20250530,2359,"51.5034938,-0.07962099","51.51772703,-0.127854211"
1,2025-01-31 23:59,1142,988,CLASSIC,36,winter,workday,20250530,2359,"51.5034938,-0.07962099","51.51772703,-0.127854211"
2,2025-01-31 23:59,300061,300024,CLASSIC,20,winter,workday,20250530,2359,"51.495598,-0.078893","51.53658514,-0.075885686"
3,2025-01-31 23:59,300058,300076,CLASSIC,4,winter,workday,20250530,2359,"51.48512191,-0.174971902","51.4795738,-0.17903854"
4,2025-02-01 00:00,300038,300037,CLASSIC,12,winter,weekend,20250531,0000,"51.48724429,-0.205279052","51.49422354,-0.236769936"


In [61]:
def get_sample(data, fraction, cols=['Bike model', 'season', 'Type of day']):
    """
        data - your dataset from which you want the sample,
        fraction - percentage of the dataset you want in decimal format (x/100)
        cols - columns that need to be ensured to survive with some proportions
    """
    data_sample = data.groupby(cols, group_keys=False).sample(frac=fraction, random_state=42)

    return data_sample


SAMPLE_SIZE = 0.1
data_sample = get_sample(data=merged_data, fraction=SAMPLE_SIZE)



print(len(merged_data))
print(len(data_sample))

data_sample.to_csv(f'../data/processed/cleaned_data_sample{int(SAMPLE_SIZE * 100)}.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)


828831
82883


Checking the data

In [62]:
import pandas as pd
data2 = pd.read_csv(f'../data/processed/cleaned_data_sample10.csv')
for col in ['Bike model', 'season', 'Type of day']:
    value_counts = data2[col].value_counts(normalize=True, dropna=False)
    print(f"\nColumn: {col}")
    print(value_counts.round(3))


Column: Bike model
Bike model
CLASSIC       0.851
PBSC_EBIKE    0.149
Name: proportion, dtype: float64

Column: season
season
summer    0.587
winter    0.413
Name: proportion, dtype: float64

Column: Type of day
Type of day
workday    0.808
weekend    0.192
Name: proportion, dtype: float64
